In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
# Define paths
train_dir = 'Dataset/Train'  # Replace with your train folder path
test_dir = 'Dataset/Test'    # Replace with your test folder path

# Image dimensions
img_width, img_height = 128, 128
input_shape = (img_width, img_height, 3)

# Batch size and epochs
batch_size = 32
epochs = 20

# Limit the number of images per class
max_images_per_class_train = 20000  # Set to None to load all images
max_images_per_class_test = 15000    # Set to None to load all images

In [3]:
# Function to load images from a folder with a limit
def load_images_from_folder(folder, max_images_per_class=None):
    images = []
    labels = []
    class_names = sorted(os.listdir(folder))  # Get subfolder names (Corrected, Reversal, Normal)
    for class_idx, class_name in enumerate(class_names):
        class_folder = os.path.join(folder, class_name)
        if os.path.isdir(class_folder):
            image_count = 0
            for filename in os.listdir(class_folder):
                if max_images_per_class and image_count >= max_images_per_class:
                    break  # Stop loading if the limit is reached
                img_path = os.path.join(class_folder, filename)
                try:
                    img = Image.open(img_path).convert('RGB')  # Ensure RGB format
                    img = img.resize((img_width, img_height))  # Resize image
                    img = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
                    if img.shape == (img_width, img_height, 3):  # Ensure 3 channels (RGB)
                        images.append(img)
                        labels.append(class_idx)  # Assign label based on subfolder
                        image_count += 1
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")
    return np.array(images), np.array(labels)

In [4]:
# Load training data with a limit
X_train, y_train = load_images_from_folder(train_dir, max_images_per_class=max_images_per_class_train)

# Load testing data with a limit
X_test, y_test = load_images_from_folder(test_dir, max_images_per_class=max_images_per_class_test)

In [5]:
# Convert labels to categorical (one-hot encoding)
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

In [6]:
# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [7]:
# Print dataset shapes
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (48000, 128, 128, 3)
Validation data shape: (12000, 128, 128, 3)
Testing data shape: (45000, 128, 128, 3)


In [8]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: Corrected, Reversal, Normal
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

c:\Users\MSI\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,940,355 (49.36 MB)

 Trainable params: 12,940,355 (49.36 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val)
)

Epoch 1/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 236s 155ms/step - accuracy: 0.9100 - loss: 0.2190 - val_accuracy: 0.9913 - val_loss: 0.0250
Epoch 2/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 237s 158ms/step - accuracy: 0.9933 - loss: 0.0217 - val_accuracy: 0.9947 - val_loss: 0.0149
Epoch 3/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 238s 159ms/step - accuracy: 0.9959 - loss: 0.0123 - val_accuracy: 0.9952 - val_loss: 0.0123
Epoch 4/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 238s 158ms/step - accuracy: 0.9973 - loss: 0.0089 - val_accuracy: 0.9948 - val_loss: 0.0142
Epoch 5/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 238s 159ms/step - accuracy: 0.9974 - loss: 0.0075 - val_accuracy: 0.9935 - val_loss: 0.0172
Epoch 6/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 237s 158ms/step - accuracy: 0.9984 - loss: 0.0055 - val_accuracy: 0.9962 - val_loss: 0.0126
Epoch 7/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 239s 159ms/step - accuracy: 0.9985 - loss: 0.0039 - val_accuracy: 0.9967 - val_loss: 0.0147
Epoch 8/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 238s 159ms/step - ac

In [10]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

1407/1407 ━━━━━━━━━━━━━━━━━━━━ 42s 29ms/step - accuracy: 0.9228 - loss: 5.2324
Test Accuracy: 77.20%


In [11]:
model.save('dyslexia_handwriting_model.h5')